### Training Word2vec From Brown Dataset 

In [5]:
import gensim
from nltk.corpus import brown
# import logging
# logging.basicConfig(
#     format='%(levelname)s : %(message)s', level=logging.INFO)
dim = [25, 100, 300]
wind = [2, 7, 15]
neg = [1, 7, 20]
for i in dim:
    for j in wind:  
        for k in neg:
            print(i,j,k)
            model = gensim.models.Word2Vec(brown.sents(),
                                            vector_size=i,
                                            window=j,
                                            negative=k,
                                            epochs=5
                                            )
            model.save('./model/brown_d(%d)_w(%d)_n(%d).embedding' %(i,j,k))

INFO : collecting all words and their counts
INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


25 2 1


INFO : PROGRESS: at sentence #10000, processed 219770 words, keeping 23488 word types
INFO : PROGRESS: at sentence #20000, processed 430477 words, keeping 34367 word types
INFO : PROGRESS: at sentence #30000, processed 669056 words, keeping 42365 word types
INFO : PROGRESS: at sentence #40000, processed 888291 words, keeping 49136 word types
INFO : PROGRESS: at sentence #50000, processed 1039920 words, keeping 53024 word types
INFO : collected 56057 word types from a corpus of 1161192 raw words and 57340 sentences
INFO : Creating a fresh vocabulary
INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 15173 unique words (27.07% of original 56057, drops 40884)', 'datetime': '2023-04-12T17:31:41.305785', 'gensim': '4.3.0', 'python': '3.10.9 | packaged by Anaconda, Inc. | (main, Mar  1 2023, 18:18:15) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'prepare_vocab'}
INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 leaves 109508

KeyboardInterrupt: 

In [8]:
import itertools
from gensim.models.word2vec import Text8Corpus
# from glove import Glove
from mittens import GloVe
from nltk.corpus import brown

ImportError: cannot import name 'Corpus' from 'mittens' (c:\Users\meghd\Anaconda3\envs\nlp\lib\site-packages\mittens\__init__.py)

In [7]:
sentences = list(itertools.islice(Text8Corpus(brown.sents()),None))

TypeError: don't know how to handle uri [['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ...]

Rounding

In [73]:
def evaluation(model_evalu):
    acc = round(model_evalu[0][0],6)
    acc = acc*100
    return acc

### Functions For Evaluation of Bats

In [54]:
import numpy as np
from numpy.linalg import norm
import random
import os

def collect(model):
    if type(model) is dict:
        vocab = [k for k in model.keys()]
    else:
        vocab = [k for k in model.wv.key_to_index.keys()]

    indices = {}
    for i in range(len(vocab)): indices[vocab[i]] = i
        
    matrix = []
    for w in vocab:
        matrix.append(model.wv[w])
    return np.array(matrix), vocab, indices

def eval_bats_file(model, matrix, vocab, indices, f, repeat=False, multi=0):
    pairs = [line.strip().split() for line in open(f, 'r').readlines()]

    # discard pairs that are not in our vocabulary
    pairs = [[p[0], p[1].split('/')] for p in pairs if p[0] in model.wv]
    pairs = [[p[0], [w for w in p[1] if w in model.wv]] for p in pairs]
    pairs = [p for p in pairs if len(p[1]) > 0]
    if len(pairs) <= 1: return None

    transposed = np.transpose(np.array([x / norm(x) for x in matrix]))
    
    if not multi:
        qa = []
        qb = []
        qc = []
        targets = []
        exclude = []
        groups = []
        for i in range(len(pairs)):
            j = random.randint(0, len(pairs) - 2)
            if j >= i: j += 1
            a = model.wv[pairs[i][0]]
            c = model.wv[pairs[j][0]]
            for bw in pairs[i][1]:
                qa.append(a)
                qb.append(model.wv[bw])
                qc.append(c)
                groups.append(i)
                targets.append(pairs[j][1])
                exclude.append([pairs[i][0], bw, pairs[j][0]])

        for queries in [qa, qb, qc]:
            queries = np.array([x / norm(x) for x in queries])
        sa = np.matmul(qa, transposed) + .0001
        sb = np.matmul(qb, transposed)
        sc = np.matmul(qc, transposed)
        sims = sb + sc - sa
        # exclude original query words from candidates
        for i in range(len(exclude)):
            for w in exclude[i]:
                sims[i][indices[w]] = 0

    else:
        offsets = []
        exclude = []
        preds = []
        targets = []
        groups = []
        
        for i in range(len(pairs) // multi):
            qa = [pairs[j][0] for j in range(len(pairs)) if j - i not in range(multi)]
            qb = [[w for w in pairs[j][1] if w in model] for j in range(len(pairs)) if j - i not in range(multi)]
            qbs = []
            for ws in qb: qbs += ws
            a = np.mean([model[w] for w in qa], axis=0)
            b = np.mean([np.mean([model[w] for w in ws], axis=0) for ws in qb], axis=0)
            a = a / norm(a)
            b = b / norm(b)

            for k in range(multi):
                c = model[pairs[i + k][0]]
                c = c / norm(c)
                offset = b + c - a
                offsets.append(offset / norm(offset))
                targets.append(pairs[i + k][1])
                exclude.append(qa + qbs + [pairs[i + k][0]])
                groups.append(len(groups))

        print(np.shape(transposed))
        sims = np.matmul(np.array(offsets), transposed)
        print(np.shape(sims))
        for i in range(len(exclude)):
            for w in exclude[i]:
                sims[i][indices[w]] = 0

    preds = [vocab[np.argmax(x)] for x in sims]
    accs = [1 if preds[i].lower() in targets[i] else 0 for i in range(len(preds))]
    regrouped = np.zeros(np.max(groups) + 1)
    for a, g in zip(accs, groups):
        regrouped[g] = max(a, regrouped[g])
    return np.mean(regrouped)

def eval_bats(model, matrix, vocab, indices):
    accs = {}
    base = './evaluation/bats'
    for dr in os.listdir('./evaluation/bats'):
        if os.path.isdir(os.path.join(base, dr)):
            dk = dr.split('_', 1)[1].lower()
            accs[dk] = []
            for f in os.listdir(os.path.join(base, dr)):
                accs[f.split('.')[0]] = eval_bats_file(model, matrix, vocab, indices, os.path.join(base, dr, f))
                accs[dk].append(accs[f.split('.')[0]])
            accs[dk] = [a for a in accs[dk] if a is not None]
            accs[dk] = np.mean(accs[dk]) if len(accs[dk]) > 0 else None

    accs['total'] = np.mean([accs[k] for k in accs.keys() if accs[k] is not None])

    return accs

## Evaluation of Bats

In [60]:
def evalu_bats(model):
    bats_cat_1 = "lexicographic_semantics"
    bats_cat_2 = "E10 [male - female]"
    bats_cat_3 = "total"

    matrix, vocab, indices = collect(model)
    bats = eval_bats(model, matrix, vocab, indices)
    bat1 = round(bats[bats_cat_1],6)
    bat2 = round(bats[bats_cat_2],6)
    bat3 = round(bats[bats_cat_3],6)
    result_bats = [bat1,bat2,bat3]
    
    return result_bats

### Initialing Tabel Headers

In [66]:
header = ['Algorithm', 'Win', 'Dim', 'NS','wordsim353','BAT1','BAT2','Bat3','win353']

# prepare accuracy of models

In [74]:
import gensim
data = []
dim = [25, 100, 300]
wind = [2, 7, 15]
neg = [1, 7, 20]
for i in dim:
    for j in wind:
        for k in neg:
            new_model = gensim.models.Word2Vec.load('./model/brown_d(%d)_w(%d)_n(%d).embedding' %(i,j,k))
            model_evalu_wordsim353= new_model.wv.evaluate_word_pairs('./evaluation/wordsim353.tsv')
            model_evalu_win353= new_model.wv.evaluate_word_pairs('./evaluation/win353.tsv')
            bats_list = evalu_bats(new_model)
            data.append(['Word2Vec',i,j,k,evaluation(model_evalu_wordsim353),bats_list[0],bats_list[1],bats_list[2],evaluation(model_evalu_win353)],)
            print(i,j,k,"=>","wordsim353: ",evaluation(model_evalu_wordsim353),
                  " win353: ",evaluation(model_evalu_win353),
                  " BAT1: ",bats_list[0],
                  "BAT2 ",bats_list[1],
                  "BAT3",bats_list[2])

25 2 1 => wordsim353:  13.0802  win353:  12.2082  BAT1:  0.025163 BAT2  0.285714 BAT3 0.02938
25 2 7 => wordsim353:  13.0435  win353:  12.2053  BAT1:  0.022627 BAT2  0.214286 BAT3 0.030254
25 2 20 => wordsim353:  13.05  win353:  12.185  BAT1:  0.031017 BAT2  0.142857 BAT3 0.028315
25 7 1 => wordsim353:  12.8822  win353:  12.1052  BAT1:  0.040281 BAT2  0.214286 BAT3 0.033563
25 7 7 => wordsim353:  12.778500000000001  win353:  12.0117  BAT1:  0.011525 BAT2  0.285714 BAT3 0.025896
25 7 20 => wordsim353:  13.1352  win353:  12.3628  BAT1:  0.030755 BAT2  0.142857 BAT3 0.035561
25 15 1 => wordsim353:  12.97  win353:  12.0762  BAT1:  0.031715 BAT2  0.214286 BAT3 0.032998
25 15 7 => wordsim353:  13.013  win353:  12.1969  BAT1:  0.03183 BAT2  0.285714 BAT3 0.025944
25 15 20 => wordsim353:  12.958300000000001  win353:  12.1617  BAT1:  0.022838 BAT2  0.071429 BAT3 0.018002
100 2 1 => wordsim353:  12.948699999999999  win353:  12.1829  BAT1:  0.045067 BAT2  0.142857 BAT3 0.038776
100 2 7 => wordsim

## Make accuracy as csv file

In [75]:
import csv
with open('./result/models_results.csv', 'w') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)

    # write the data
    writer.writerows(data)

### Make table

In [76]:
import plotly.graph_objects as go
import pandas as pd
# ['Algorithm', 'Win', 'Dim', 'NS','wordsim353','BAT1','BAT2','Bat3','win353']
df = pd.read_csv('./result/models_results.csv')
fig = go.Figure(data=[go.Table(
    header=dict(values=list(df.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df.Algorithm, df.Win, df.Dim, df.NS, df.wordsim353,df.BAT1,df.BAT2,df.Bat3, df.win353],
               fill_color='lavender',
               align='left'))
])

fig.show()

# THE END

###################################################################################

# Editional Codes

In [ ]:
# one time used 
# convert csv to tsv
import re
 
# reading given tsv file
with open("win353.csv", 'r') as myfile: 
  with open("win353.tsv", 'w') as csv_file:
    for line in myfile:
       
      # Replace every tab with comma
      fileContent = re.sub(",", "\t", line)
       
      # Writing into csv file
      csv_file.write(fileContent)
 
# output
print("Successfully made tsv file")

In [ ]:
model = gensim.models.Word2Vec(brown.sents())
model.save('./model/brown.embedding')
new_model = gensim.models.Word2Vec.load('./model/brown.embedding')

In [3]:
import gensim
new_model = gensim.models.Word2Vec.load('./model/brown.embedding')

In [10]:
import numpy as np
X = np.zeros(shape=(50,len(new_model.wv['university'])))
X.shape

(50, 300)

In [ ]:
new_model.wv.most_similar("university")

In [ ]:
from gensim.test.utils import datapath
model_evalu= new_model.wv.evaluate_word_pairs(datapath('wordsim353.tsv'))
print(float(model_evalu[2]))

In [ ]:
CSV_NAME = 'models_eval.csv'

bats_cat_1 = "lexicographic_semantics"
bats_cat_2 = "E10 [male - female]"
bats_cat_3 = "total"